# Pipeline for ME740 Project done by Sam Hawkins
## I recommend using Google Colaboratory to run this notebook, as there are many different downloads of packages and requirement files. This notebook can be extremely computationally expensive.
## To run the code and view your own results, simply run the cells in order. If you need to reduce computational complexity, reduce the "resize" option in the LFW dataset import to a lower fraction
### Open-Source repositories used in this notebook directly: https://github.com/xinntao/BasicSR, https://github.com/serengil/deepface, https://github.com/TencentARC/GFPGAN 

# First Step: ESRGAN Inference -> Super Resolution of LFW dataset using ESRGAN


# Preparations
Before start, make sure that you choose
* Runtime Type = Python 3
* Hardware Accelerator = GPU

in the **Runtime** menu -> **Change runtime type**


## Git clone BasicSR repo which contains ESRGAN model and requirements

In [ ]:
!rm -rf BasicSR
!git clone https://github.com/xinntao/BasicSR.git
%cd BasicSR

## Set up the enviroment


In [ ]:
# Install pytorch
!pip install torch torchvision

# Check torch and cuda versions
import torch
print('Torch Version: ', torch.__version__)
print('CUDA Version: ', torch.version.cuda)
print('CUDNN Version: ', torch.backends.cudnn.version())
print('CUDA Available:', torch.cuda.is_available())

In [ ]:
# Install requirements
!pip install -r requirements.txt
# Install BasicSR without cuda extentions
!python setup.py develop

## Download pretrained models

In [ ]:
!python scripts/download_pretrained_models.py ESRGAN

## Set proper folders in directory to place all necessary images inside

### Create folders in directory

In [ ]:
import os
from google.colab import files
import shutil

os.makedirs('results/ESRGAN', exist_ok=True)

upload_folder = 'datasets/upload'
result_folder = 'results/ESRGAN'
compare_folder = 'results/interpolation'
LR_folder = 'datasets/LR'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
if os.path.isdir(compare_folder):
    shutil.rmtree(compare_folder)
if os.path.isdir(LR_folder):
    shutil.rmtree(LR_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)
os.mkdir(compare_folder)
os.mkdir(LR_folder)

"""
# upload images
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)
"""

# Testing Phase 1 - Perform Super Resolution on LFW pairs, compare recognition rates for original, Bicubic Interpolation, and for ESRGAN

### Get requirements and packages

In [ ]:
#Face recognition model install
!pip install deepface

In [ ]:
from deepface import DeepFace
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from tqdm import tqdm
import os
import glob
import cv2
import numpy as np
import torch

from basicsr.archs.rrdbnet_arch import RRDBNet

Directory  /root /.deepface created
Directory  /root /.deepface/weights created


## Import LFW dataset.
### Run as is for color images. If you want to test on Black and White images, set color to False. If doing Black and White testing, uncomment the commented lines the have /BW at the end and comment the corresponding color counterpart code line
### Using the training subset for more images

In [ ]:
from sklearn.datasets import fetch_lfw_pairs
#fetch_lfw_pairs(subset='train',data_home=None,funneled=True
#,resize=0.5,color=False,slice_=(slice(70,195),slice(78,172)),download_if_missing=True)
#original img size (250,250), size with default resize and slice (62,47)
fetch_lfw_pairs = fetch_lfw_pairs(subset = 'train', funneled=True
, color = True, resize = .5)

## Define pairs
### For 'test', there are 1000 images, first 500 have same person pairs, second 500 have different people pairs. For 'train', 2200 images, 1100 same, 1100 different

In [ ]:
pairs = fetch_lfw_pairs.pairs
labels = fetch_lfw_pairs.target
target_names = fetch_lfw_pairs.target_names

In [ ]:
pairs.shape

In [ ]:
all_img = pairs.reshape(pairs.shape[0]*2,pairs.shape[2],pairs.shape[3],pairs.shape[4])
#all_img = pairs.reshape(pairs.shape[0]*2,pairs.shape[2],pairs.shape[3])#BW

### Add LR images to LR folder in directory

In [ ]:
for i in range(len(all_img)):
    img = all_img[i]
    cv2.imwrite(f'datasets/LR/{i}.png',img)

### SR by Bicubic Interpolation of LR images

In [ ]:
folder = 'datasets/LR'

os.makedirs('results/interpolation', exist_ok=True)
for idx, path in enumerate(sorted(glob.glob(os.path.join(folder, '*')))):
    imgname = os.path.splitext(os.path.basename(path))[0]
    #print(idx, imgname)
    # read image
    img = cv2.imread(path, cv2.IMREAD_COLOR).astype(np.float32) / 255.
    #img = cv2.imread(path, cv2.IMREAD_GRAYSCALE).astype(np.float32) / 255. #BW


    # inference
    output = cv2.resize(img,(img.shape[1]*4,img.shape[0]*4))

    # save image
    #output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round().astype(np.uint8)

    cv2.imwrite(f'results/interpolation/{imgname}_interp.png', output)

### SR by ESRGAN of LR images

In [ ]:
import cv2
import glob
import numpy as np
import os
import torch


from basicsr.archs.rrdbnet_arch import RRDBNet

# configuration
model_path = 'experiments/pretrained_models/ESRGAN/ESRGAN_SRx4_DF2KOST_official-ff704c30.pth'

folder = 'datasets/LR'
device = 'cuda'

device = torch.device(device)

# set up model
model = RRDBNet(
    num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32)
model.load_state_dict(torch.load(model_path)['params'], strict=True)
model.eval()
model = model.to(device)

os.makedirs('results/ESRGAN', exist_ok=True)
for idx, path in enumerate(sorted(glob.glob(os.path.join(folder, '*')))):
    imgname = os.path.splitext(os.path.basename(path))[0]
    #print(idx, imgname)
    # read image
    img = cv2.imread(path, cv2.IMREAD_COLOR).astype(np.float32) / 255.
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]],
                                        (2, 0, 1))).float()
    img = img.unsqueeze(0).to(device)
    # inference
    with torch.no_grad():
        output = model(img)
    # save image
    output = output.data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round().astype(np.uint8)
    cv2.imwrite(f'results/ESRGAN/{imgname}_ESRGAN.png', output)

### Download Results (OPTIONAL) - Uncomment if you want zip files of results

In [ ]:
# download the ESRGAN result
"""print(f'Download {result_folder}')
os.system(f'zip -r -j download.zip {result_folder}/*')
files.download("download.zip")"""

In [ ]:
# download the bicubic
"""
print(f'Download {compare_folder}')
os.system(f'zip -r -j bicubic.zip {compare_folder}/*')
files.download("bicubic.zip")"""

## Get all three sets of results back into pairs

### Extract images from result folders into variables

In [ ]:
import natsort
#sorted = natsort.natsorted(os.listdir(path))
def extract_results(path,all_lr):

    #images = np.zeros((all_lr.shape[0],all_lr.shape[1]*4,all_lr.shape[2]*4,all_lr.shape[3]))
    images = []

    for file in natsort.natsorted(os.listdir(path)):
        img = os.path.join(path,file)
        img = cv2.imread(img,cv2.IMREAD_COLOR)
        #img = cv2.imread(img,cv2.IMREAD_GRAYSCALE)#BW
        #images[i] = img
        images.append(img)

    images = images[:all_lr.shape[0]] #in order to save RAM if necessary
    return images

In [ ]:
interp_results = np.array(extract_results('results/interpolation',all_img))
esrgan_results = np.array(extract_results('results/ESRGAN',all_img))

In [ ]:
interp_results.shape

# Testing Phase 2: Run Face Optimized GAN model (GFPGAN) and extract results

## Import Model

In [ ]:
# Clone GFPGAN and enter the GFPGAN folder
%cd /content
!rm -rf GFPGAN
!git clone https://github.com/TencentARC/GFPGAN.git
%cd GFPGAN

# Set up the environment
# Install basicsr - https://github.com/xinntao/BasicSR
# We use BasicSR for both training and inference

# Install facexlib - https://github.com/xinntao/facexlib
# We use face detection and face restoration helper in the facexlib package
!pip install facexlib
# Install other depencencies
!pip install -r requirements.txt
!python setup.py develop
!pip install realesrgan  # used for enhancing the background (non-face) regions
# Download the pre-trained model
# !wget https://github.com/TencentARC/GFPGAN/releases/download/v0.2.0/GFPGANCleanv1-NoCE-C2.pth -P experiments/pretrained_models
# Now we use the V1.3 model for the demo
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models


### Inference

In [ ]:
os.makedirs('datasets/LR',exist_ok=True)
for i in range(len(all_img)):
    img = all_img[i]
    cv2.imwrite(f'datasets/LR/{i}.png',img)

In [ ]:
# Now we use the GFPGAN to restore the above low-quality images
# We use [Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN) for enhancing the background (non-face) regions
# You can find the different models in https://github.com/TencentARC/GFPGAN#european_castle-model-zoo

!rm -rf results
!python inference_gfpgan.py -i datasets/LR -o results -v 1.3 -s 4 --bg_upsampler realesrgan --suffix gfpgan --ext png

# Usage: python inference_gfpgan.py -i inputs/whole_imgs -o results -v 1.3 -s 2 [options]...
# 
#  -h                   show this help
#  -i input             Input image or folder. Default: inputs/whole_imgs
#  -o output            Output folder. Default: results
#  -v version           GFPGAN model version. Option: 1 | 1.2 | 1.3. Default: 1.3
#  -s upscale           The final upsampling scale of the image. Default: 2
#  -bg_upsampler        background upsampler. Default: realesrgan
#  -bg_tile             Tile size for background sampler, 0 for no tile during testing. Default: 400
#  -suffix              Suffix of the restored faces
#  -only_center_face    Only restore the center face
#  -aligned             Input are aligned faces
#  -ext                 Image extension. Options: auto | jpg | png, auto means using the same extension as inputs. Default: auto

!ls results/cmp

In [ ]:
gfpgan_results = np.array(extract_results('results/restored_imgs',all_img))

## Check to make sure the orders align, and visualize some example results
### Functions to visualize

In [ ]:
def plot_results(orig,interp,esrgan,gfpgan):
    fig = plt.figure(figsize=(25, 10))
    ax1 = fig.add_subplot(1, 4, 1) 
    plt.title('Input image', fontsize=16)
    ax1.axis('off')
    ax2 = fig.add_subplot(1, 4, 2)
    plt.title('Bicubic Interpolation', fontsize=16)
    ax2.axis('off')
    ax3 = fig.add_subplot(1, 4, 3)
    plt.title('ESRGAN output', fontsize=16)
    ax3.axis('off')
    ax4 = fig.add_subplot(1, 4, 4)
    plt.title('GFPGAN output', fontsize=16)
    ax4.axis('off')
    ax1.imshow(orig/255)
    ax2.imshow(interp)
    ax3.imshow(esrgan)
    ax4.imshow(gfpgan)
    
    #ax1.imshow(orig,cmap='gray',vmin=0,vmax=255)
    #ax2.imshow(interp,cmap='gray',vmin=0,vmax=255)
    #ax3.imshow(esrgan,cmap='gray',vmin=0,vmax=255)
    #ax4.imshow(gfpgan,cmap='gray',vmin=0,vmax=255)
    

def plot_rand(num,orig,interp,esrgan,gfpgan):
    for i in range(num):
        randy = np.random.randint(len(interp))
        plot_results(orig[randy],interp[randy],esrgan[randy],gfpgan[randy])

## Visualize some images
### To change number of images to view, change number to the amount of result comparisons you want to see

In [ ]:
plot_rand(40,all_img,interp_results,esrgan_results,gfpgan_results)

### Transform all sets back into pairs

In [ ]:
def reshape_pairs(images):
    #Note that there needs to be even number in original array, as we need pairs
    reshaped = images.reshape(int(images.shape[0]/2), 2, images.shape[1], images.shape[2], images.shape[3])
    #reshaped = images.reshape(int(images.shape[0]/2), 2, images.shape[1], images.shape[2])#BW

    return reshaped

In [ ]:
esrgan_pairs = reshape_pairs(esrgan_results)
interp_pairs = reshape_pairs(interp_results)
gfpgan_pairs = reshape_pairs(gfpgan_results)

## (OPTIONAL) Plot the pairs, ensuring the same people are in pairs from different image sets
### This shows the pairs and the ground truth of whether or not it's the same person

In [ ]:
def plot_pairs(num, pairs, labels):

    for i in range(0, num):
      pair = pairs[i]
      img1 = pair[0]
      img2 = pair[1]
    
      fig = plt.figure()
    
      ax1 = fig.add_subplot(1,3,1)
      plt.imshow(img1/255)
    
      ax2 = fig.add_subplot(1,3,2)
      plt.imshow(img2/255)
    
      ax3 = fig.add_subplot(1,3,3)
      plt.text(0, 0.50, target_names[labels[i]])
    
      plt.show()

In [ ]:
"""plot_pairs(5,esrgan_pairs, labels)
plot_pairs(5,interp_pairs, labels)
plot_pairs(5,pairs, labels)
plot_pairs(5, gfpgan_pairs, labels)"""

# Final Step: Compare Facial Recognition accuracy on the three sets
## Import Facenet model from deepface framework
### Run recognition to tell whether or not the pairs have the same or different people

In [ ]:
def recognition_predictions(img_pairs, model_name = 'Facenet'):

    #model_name options: (string) VGG-Face, Facenet, OpenFace, DeepFace, DeepID, Dlib, ArcFace or Ensemble
    #img_pairs is images we are comparing: dimensions(n-pairs,2, img_h,img_w,#channels)
    model = DeepFace.build_model(model_name)
    predictions = []

    #If Black and white
    #img_pairs = np.stack((img_pairs,)*3, axis=-1)  

    for idx in tqdm(range(0,img_pairs.shape[0])):

        pair = img_pairs[idx]
        img1 = pair[0]
        img2 = pair[1]

        #actual = labels[idx]
        obj = DeepFace.verify(img1, img2, model_name = model_name,
                              model=model, enforce_detection = False,
                              detector_backend = 'opencv')
        
        prediction = 1 if obj["verified"] == True else 0

        predictions.append(prediction)

    return predictions


## Extract results into variables - This step takes the longest

In [ ]:
orig_recog = recognition_predictions(pairs, model_name='Facenet')
interp_recog = recognition_predictions(interp_pairs, model_name='Facenet')
esrgan_recog = recognition_predictions(esrgan_pairs, model_name='Facenet')
gfpgan_recog = recognition_predictions(gfpgan_pairs, model_name='Facenet')

## Determine results of recognition

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

def compute_performance(actuals,predictions):

    #Accuracy, precision, recall, F1
    accuracy = 100*accuracy_score(actuals, predictions)
    precision = 100*precision_score(actuals, predictions)
    recall = 100*recall_score(actuals, predictions)
    f1 = 100*f1_score(actuals, predictions)
    print("Accuracy = ",accuracy,"%")
    print("Precision = ",precision,"%")
    print("Recall = ",recall,"%")
    print("F1 Score = ",f1,"%")

    #Confusion matrix
    cm = confusion_matrix(actuals, predictions)
    print("\nConfusion Matrix:\n",cm)
    
    #Numbers of true/false positives/negatives from CM
    tn, fp, fn, tp = cm.ravel()
    print("\nTrue Negative = ",tn)
    print("False Positive = ", fp)
    print("False Negative = ",fn)
    print("True Positive = ",tp ,'\n')

    return

### Show results

In [ ]:
compute_performance(labels, orig_recog)
compute_performance(labels, interp_recog)
compute_performance(labels, esrgan_recog)
compute_performance(labels, gfpgan_recog)